In [1]:
import os
import glob
from tqdm import tqdm
import pandas as pd 
import openslide
from histoplus.extract import extract
from histoplus.helpers.segmentor import CellViTSegmentor
from histoplus.helpers.tissue_detection import detect_tissue_on_wsi

import json
# import ijson
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from PIL import Image, ImageDraw, ImageFont
from typing import Dict, Tuple, List

import math
from collections import Counter
from openslide import OpenSlide
from openslide.deepzoom import DeepZoomGenerator

/home/sjhong/miniconda3/envs/cellvit/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df['Patient_id'].tolist()

['1-054_10',
 '1-066_11',
 '1-035_09',
 '1_120_17',
 '1_160_19',
 '1-008_06',
 '1-048_10',
 '1-039_09',
 '1_115_17',
 '1-013_07',
 '1_147_19',
 '1-046_10',
 '1-007_05',
 '1-075_11',
 '1_125_17',
 '1_148_19',
 '1_126_17',
 '1-085_12',
 '1-084_12',
 '1-011_06',
 '1-015_07',
 '1_128_18',
 '1-060_11',
 '1-031_09',
 '1_121_17',
 '1-045_10',
 '1-028_08',
 '1-070_11',
 '1_136_19',
 '1-021_08',
 '1-057_11',
 '1_152_19',
 '1-077_11',
 '1-063_11',
 '1-022_08',
 '1-033_09',
 '1-040_09',
 '1_105_16',
 '1-006_04',
 '1-076_11',
 '1_129_18',
 '1-025_08',
 '1-082_12',
 '1-043_10',
 '1-055_10',
 '1-061_11',
 '1_161_19',
 '1-069_11',
 '1-032_09',
 '1-059_11',
 '1_134_18',
 '1-038_09',
 '1-009_06',
 '1-018_07',
 '1-074_11',
 '1-080_11',
 '1-027_08',
 '1_131_18',
 '1_144_19',
 '1-026_08',
 '1_153_19',
 '1-051_10',
 '1-065_11',
 '1_116_17',
 '1_122_17',
 '1-047_10',
 '1-029_08',
 '1-034_09',
 '1-068_11',
 '1-044_10',
 '1-024_08',
 '1_103_16',
 '1-005_04',
 '1_114_17',
 '1-078_11',
 '1-073_11',
 '1-017_07',

In [7]:
INFERENCE_IMAGE_SIZE = 588
MPP = 0.25

output_dir = './output/KBSMC_HEATMAP_updated/'
os.makedirs(output_dir, exist_ok = True)

df = pd.read_csv("/home/yesung/new_names.csv")
sample_list = os.listdir("/home/nas2_fast/Data/Pathology_project/KBSMC/wsi")
sample_list = [x for x in sample_list if x.split('/')[-1][:-4] in df['Patient_id'].tolist()]
print(len(sample_list))

97


In [ ]:
priority = df.iloc[[0,36,37,65],1].tolist()
last = df.loc[~df.index.isin([0,36,37,65]), df.columns[1]].tolist()
sample_names = priority + last
sample_names

In [9]:
sample_names = [word[:-4] for word in sample_list]

In [12]:
sample_list = [f'/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/{x}.svs' for x in sample_names]
print(len(sample_list))

97


In [13]:
sample_list[0], sample_list[-1]

('/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1-001_04.svs',
 '/home/nas2_fast/Data/Pathology_project/KBSMC/wsi/1_161_19.svs')

In [15]:
for  wsi_path  in tqdm(sample_list):
    
    if not os.path.exists(wsi_path):
        tqdm.write(wsi_path, "doesn't exsisted.")
        continue
    
    file_name = wsi_path.split('/')[-1][:-4]
    
    json_path = f"{output_dir}/seg_{file_name}_{INFERENCE_IMAGE_SIZE}.json"
    if os.path.exists(json_path):
        continue
    
    try:
        slide = openslide.OpenSlide(wsi_path)
        slide.dimensions, slide.properties[openslide.PROPERTY_NAME_MPP_X]
        tissue_coords, dz_level = detect_tissue_on_wsi(slide)
        # 세그멘터 로드(가중치 자동 다운로드; HF 로그인 필요)
        segmentor = CellViTSegmentor.from_histoplus(
            mpp=MPP,
            mixed_precision=True,                 # GPU AMP 사용 권장
            inference_image_size=INFERENCE_IMAGE_SIZE,
        )

        # Process a whole slide image
        results = extract(
            slide=slide,
            coords=tissue_coords,
            deepzoom_level=dz_level,
            segmentor=segmentor,
            batch_size= 64, # 786 에러
        )

        results.save(json_path)
        print(f"OK → {json_path}") # 저장도 오래걸림 case2는 1min 20sec
        
        
    except Exception as e:
        print(file_name, e)

  0%|          | 0/97 [00:00<?, ?it/s]2025-10-11 13:51:33.877 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-10-11 13:51:33.879 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
2025-10-11 13:51:38.292 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 6898 tiles with Otsu's threshold.
2025-10-11 13:51:41.336 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.261525 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.261525.
  1%|          | 1/97 [15:48<25:17:19, 948.33s/it]

OK → ./output/KBSMC_HEATMAP_updated//seg_1-001_04_588.json


2025-10-11 14:07:22.029 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.40704 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.26272.
2025-10-11 14:07:22.031 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52544 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.26272.
2025-10-11 14:07:25.309 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 4613 tiles with Otsu's threshold.
2025-10-11 14:07:27.771 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.26272 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.26272.
  2%|▏         | 2/97 [28:15<21:54:07, 829.97s/it]

OK → ./output/KBSMC_HEATMAP_updated//seg_1-002_04_588.json


2025-10-11 14:19:49.291 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-10-11 14:19:49.293 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
2025-10-11 14:19:56.707 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 6384 tiles with Otsu's threshold.
2025-10-11 14:19:59.489 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.261525 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.261525.
  3%|▎         | 3/97 [42:36<22:02:38, 844.24s/it]

...

OK → ./output/KBSMC_HEATMAP_updated//seg_1_155_19_588.json


2025-10-12 12:56:12.480 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-10-12 12:56:12.481 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
2025-10-12 12:56:18.767 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 8988 tiles with Otsu's threshold.
2025-10-12 12:56:22.431 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.261525 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.261525.
 99%|█████████▉| 96/97 [23:25:36<21:25, 1285.12s/it]

OK → ./output/KBSMC_HEATMAP_updated//seg_1_160_19_588.json


2025-10-12 13:17:10.469 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 8.3688 is the closest available MPP to the one requested (8.0). Minimal MPP of the slide is 0.261525.
2025-10-12 13:17:10.470 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.52305 is the closest available MPP to the one requested (0.5). Minimal MPP of the slide is 0.261525.
2025-10-12 13:17:17.678 | INFO     | histoplus.helpers.tissue_detection.core:detect_tissue_on_wsi:80 - Found 3919 tiles with Otsu's threshold.
2025-10-12 13:17:20.925 | INFO     | histoplus.helpers.tiling.optimal_mpp:get_tiling_slide_level:149 - 0.261525 is the closest available MPP to the one requested (0.25). Minimal MPP of the slide is 0.261525.
100%|██████████| 97/97 [23:35:13<00:00, 875.40s/it] 

OK → ./output/KBSMC_HEATMAP_updated//seg_1_161_19_588.json
